### 이미지 기반 질의 응답

<br>

[외부 지식 기반 멀티모달 질의응답 데이터](https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=713570)
<br>[이미지 기반 질의 응답](https://aifactory.space/task/2652/overview)

<br>

In [ ]:
!pip install gdown
!pip install torch==1.12.1
!pip install torchaudio==0.12.1
!pip install torchvision==0.13.1
!pip install transformers==4.21.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 2.3 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.1.0+cu118
    Uninstalling torch-2.1.0+cu118:
      Successfully uninstalled torch-2.1.0+cu118
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu118 requires torch==2.1.0, but you have torch 1.12.1 which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 1.12.1 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 1.12.1 which is incompatible.
torchvision 0.16.0+cu118 requires torch==2.1.0, but you have torch 1.12.1 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 31.6 MB/s eta 0:00:00
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.1.0+cu118
    Uninstalling torchaudio-2.1.0+cu11

In [2]:
import sys
import os
import torch
import argparse
import pandas as pd

from transformers import DefaultDataCollator
from transformers import TrainingArguments
from transformers import Trainer
from utils import VQAtrainingDataset, load_dataset, load_model

In [15]:
# img_train_path = sys.argv[1]
# anno_train_path = sys.argv[2]
# model_save_path = sys.argv[3]

# sys.arvg
img_train_path = 'data/train_data/train_img'
anno_train_path = 'data/train_data/train.json'
model_save_path = 'output'

# fix
model_checkpoint = 'dandelin/vilt-b32-finetuned-vqa'
batch_size = 20
phase = 'train'
label_path = 'data/competition/vocab.txt'

if torch.cuda.is_available() == True:
    device = 'cuda'
else :
    device = 'cpu'

if not os.path.exists(model_save_path):
    os.makedirs(model_save_path)

#######
config, train_dataset = load_dataset(model_checkpoint, label_path, anno_train_path, phase)

train_dataset_df = pd.DataFrame(train_dataset)
model, processor = load_model(model_checkpoint, config)
model.to(device)
train_processed_dataset = VQAtrainingDataset(train_dataset_df, processor, img_train_path, config, phase)
data_collator = DefaultDataCollator()

training_args = TrainingArguments(
    output_dir=model_save_path,
    per_device_train_batch_size=32,
    num_train_epochs=20,
    save_total_limit=10,
    save_strategy='epoch',
    logging_steps=50,
    learning_rate=5e-5,
    remove_unused_columns=False,
)


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_processed_dataset,
    tokenizer=processor,
)

trainer.train() 

Some weights of ViltForQuestionAnswering were not initialized from the model checkpoint at dandelin/vilt-b32-finetuned-vqa and are newly initialized because the shapes did not match:
- classifier.3.weight: found shape torch.Size([3129, 1536]) in the checkpoint and torch.Size([374, 1536]) in the model instantiated
- classifier.3.bias: found shape torch.Size([3129]) in the checkpoint and torch.Size([374]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ValueError: The image to be converted to a PIL image contains values outside the range [0, 1], got [-1.9809060096740723, 2.640000104904175] which cannot be converted to uint8.

In [16]:
import os
import sys
import torch
import argparse
import pandas as pd
import json

from utils import VQAtrainingDataset, load_dataset
from transformers import ViltForQuestionAnswering, ViltProcessor
from tqdm import tqdm

In [17]:
def load_model(model_path):
    processor = ViltProcessor.from_pretrained(model_path)
    model = ViltForQuestionAnswering.from_pretrained(model_path)
    return model, processor

# model_path = sys.argv[1]
# anno_test_path = sys.argv[2]
# y_pred_save_path = sys.argv[3]

model_path = 'data/competition/model'
anno_test_path = 'data/val_data/val.json'
y_pred_save_path = 'y_pred.json'

# fix
model_checkpoint = 'dandelin/vilt-b32-finetuned-vqa'
batch_size = 20
phase = 'val'
label_path = 'data/competition/vocab.txt'
img_path = 'data/val_data/val_img'

if torch.cuda.is_available() == True:
    device = 'cuda'
else :
    device = 'cpu'

model = os.path.join(model_path, os.listdir(model_path)[-1])

model, processor = load_model(model)
model.to(device)
val_dataset = load_dataset(model, label_path, anno_test_path, phase=phase, config=model.config)
dataset_df = pd.DataFrame(val_dataset)
processed_dataset = VQAtrainingDataset(dataset_df, processor, img_path, model.config, phase)

result = {}
with torch.no_grad():
    model.eval()
    for idx in tqdm(range(len(processed_dataset)), desc='inference'):
        data = processed_dataset[idx]
        question = processor.decode(data['encoding']['input_ids']) # need postprocessing
        question_id = data['question_id']
        # label_token = data['labels']
        # label = model.config.id2label[label_token]
        example = {k: v.unsqueeze(0).to(device) for k,v in data['encoding'].items()}
        outputs = model(**example)
        logits = outputs.logits
        predicted_classes = torch.sigmoid(logits)

        # probs, classes = torch.topk(predicted_classes, 5)
        probs, classes = torch.topk(predicted_classes, 1) # top 1

        # for prob, class_idx in zip(probs.squeeze().tolist(), classes.squeeze().tolist()):
        #     print(prob, model.config.id2label[class_idx])
        pred = model.config.id2label[classes.squeeze().tolist()] 
        # dict_result = {'question_id':question_id, 
        #             'label_token':int(label_token), 'label':label, 
        #             'pred_class':classes.squeeze().tolist(), 'pred':pred} 
        result[question_id] = pred 
        # result.append(dict_result)

with open(y_pred_save_path, 'w') as f:
    json.dump(result, f, indent=4)
# #############
# model = load_model(model_path)
# x_test = np.load(x_test_path)

# y_pred = model.predict(x_test)
# y_pred_labels = np.argmax(y_pred, axis=1)

# np.save(y_pred_save_path, y_pred_labels)


inference:   0%|          | 0/400 [00:00<?, ?it/s]


ValueError: The image to be converted to a PIL image contains values outside the range [0, 1], got [-2.018207311630249, 2.4285714626312256] which cannot be converted to uint8.